In [1]:
import sympy as sy
sy.init_printing(use_latex=True)
import numpy as np
import pandas as pd
import scipy as sc
import scipy.linalg as LA
import matplotlib.pyplot as plt
import math
import vpython as vp
import scipy.integrate as integrate
import copy
vec = vp.vector

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 2. Abgabe aus dem Fach Modellbildung und Simulation - Klassische Mechanik 2

## Keplerproblem

Zwei Massen $M$ und $m$, die den Abstand $r$ zueinander besitzen, ziehen sich gegenseitig mit der Kraft
$$ \vec{F}_G = -G\frac{Mm}{r^2}\vec{e}_r $$
an. $G$ ist die universelle Gravitationskonstante.

In [ ]:
x = sy.symbols('x')

### 1.
Im File Sat.txt sind die Umlaufzeiten $T$ und mittleren Radien $r$ der Umlaufbahnen von vier Satelliten angegeben (die einen Asteroiden umkreisen). Zeigen Sie, dass sich die Datein mit der Form $T=Cr^n$ beschreiben lassen. Bestimmen Sie $n$ und $C$, indem sie Sie die Daten logarithmisch auftragen. Schätzen Sie den fehlenden Radius ab.

Aufgabe herausfinden was C und n sein können.

Die Umlaufzeit ist gegeben durch $a= \omega^2r = (\frac{2\pi}{T})^2r$

In [ ]:
with open("Sat.txt", "r") as f:
    sat = [line.split() for line in f]
header_ = sat[0]
header = []
h_str1 = ''
h_str2 = ''
for i in range(0, 3):
    h_str1 += header_[i] + ' '
    h_str2 += header_[i+3] + ' '
header = [h_str1, h_str2]
sat[0] = header
print(header)
print(sat)
sat_data = {}
for s in range(1, len(sat)):
    for n in range(0, len(sat[s])):
        if sat[s][n] == '?':
            sat[s][n] = x
        else:
            sat[s][n] = float(sat[s][n])
sat_data = {sat[0][i]: [ line[i] for line in sat[1:] ]  for i in range(len(sat[0]))}
print(sat_data)
xs = sat_data['T in a '][0:3]
xs.append(sat_data['T in a '][4])
print(xs)

ys = sat_data['r in Gm '][0:3]
ys.append(sat_data['r in Gm '][4])
print(ys)
plt.loglog(xs, ys, 'o')
plt.loglog(xs, ys)
plt.xlabel('T in a')
plt.ylabel('r in Gm')
plt.show()

$\mathrm{ln}(a) + b \mathrm{ln}(x) = \mathrm{ln}(y) $
Berechnung Potenzfunktion : $y = ax^b$

### 2.
Schreiben Sie ein Programm, das die Differentialgleichung
$$ m\ddot{\vec{r}} = -G\frac{Mm}{r^3}\vec{r} $$
löst und visualisieren Sie die Bahnkurve der Erde um die Sonne. Verwenden sie als Anfangsgeschwindigkeit $(0.0, 29.8)*10^3m/s$ und den Anfangsort der Erde $(1.4960, 0.0)*10^{11}m$ (Die Sonne ruhe anfänglich im Urspung). Überprüfen Sie die Impuls-, Drehimpuls- und Energieerhaltung numerisch.

In [ ]:
M, m, r, G, vr, t = sy.symbols('M, m, r, G, \\vec{r}, t')
ddr = sy.symbols('\ddot{\\vec{r}}', function=True)

In [ ]:
gl = sy.Eq(m*ddr(t), -G*((M*m)/r**3)*vr)
display(gl)
gl = sy.solve(gl, ddr(t))
display(gl)
gl2 = sy.Eq(ddr(t), gl[0])
display(gl2)

In [ ]:
def rk4(y, t, m, g, h, ode):
    k1 = np.array(ode(t, y, m, g))
    k2 = np.array(ode(t+(h/2), y+ (h/2)*k1, m, g))
    k3 = np.array(ode(t+(h/2), y+(h/2)*k2, m, g))
    k4 = np.array(ode(t+h, y+h*k3, m, g))
    ynew = y + h*(k1+2*k2+2*k3+k4)/6
    return ynew

In [ ]:
def ODEPlaneten_rk(t, y, massen, G_val):
    '''
    berechnet die rechte Seite der Differentialgleichung 
    Zunächst kommen alle Variablen (x,y,vx,vy) für den ersten Planeten, dann alle 4 für den zweiten,... 
    :param t: aktueller zeitpunkt
    :param y: aktueller y-Wert (n Komponenten)
    :return: rechte Seite f der Dgl. y'=f(t,y)
    '''

    G = G_val #Gravitationskonstante
    y = np.array(y)
    y = y.reshape(-1, 4)
    rs = []
    dy = []
    for i in range(0, len(y)):
        F_i = np.array([0., 0.])
        rs.append(np.array([y[i][0], y[i][1]]))
        r_i = np.array([y[i][0], y[i][1]])
        for j in range(0, len(y)):
            if i != j:
                r_j = np.array([y[j][0], y[j][1]])
                m_j = massen[j]
                dist = r_i-r_j
                if LA.norm(dist) > 0:
                    F_i += m_j *((dist)/(LA.norm(dist))**3)
                else:
                    F_i += np.array([0., 0.])
        ddr_i = -G*F_i
        dy.append(y[i][2])
        dy.append(y[i][3])
        dy.append(ddr_i[0])
        dy.append(ddr_i[1])
    return dy

In [ ]:
def integrateall(method, ode, massen, G_val, y0, t0, tend, N, intermediate=False):
    """
    :param method: Enschrittverfahren
    :param ode: Rechte seite der Dgl
    :param y0: Anfangswert
    :param t0: Anfangszeitpunkt
    :param tend: Endzeitpunkt
    :param N: Anzahl der Schritte
    :param intermediate: True, wenn zum Plotten der Lösung auch die Ergebnisse der Zwischenschritte zurückgegeben werden sollen
    :return: Lösung am Endzeitpunkt (bzw. zusätzlich an den Zwischenzetpunkten, wenn intermediate =True)
    """
    yvals = [y0]
    space =  np.linspace(t0, round(tend, 5), num=N, retstep=True, dtype=float)
    tvals = space[0]
    h = space[1]
    y = y0
    for ts in tvals:
        y = method(y, ts, massen, G_val, h, ode)
        yvals.append(y)
    del yvals[-1]
    yend=yvals[-1]
    if intermediate:
        return np.array(yvals), tvals
    else:
        return yend

In [ ]:
def Planeten_Coordinates(Planeten, g, t0, tend, N):
    Planet_n = []
    tvals, h = np.linspace(t0, tend, N, retstep=True)
    coordinates = {}
    y_planeten = []
    massen = []
    G_val = g
    for planet in Planeten:
        Planet_n.append(planet)
        x_s = Planeten[planet]['pos'][0]
        y_s = Planeten[planet]['pos'][1]
        dx_s = Planeten[planet]['gesch'][0]
        dy_s = Planeten[planet]['gesch'][1]
        y_planeten.append(x_s)
        y_planeten.append(y_s)
        y_planeten.append(dx_s)
        y_planeten.append(dy_s)
        massen.append(Planeten[planet]['masse'])
        coordinates[planet] = {'x':[], 'y':[]}
    y_list, t_list = integrateall(rk4, ODEPlaneten_rk, massen, G_val, y_planeten, t0, tend, N, intermediate=True)
    for y_step in y_list:
        y_shaped = y_step.reshape(-1, 4)
        for i in range(0, len(y_shaped)):
            list_c = coordinates[Planet_n[i]]['x']
            list_c.append(y_shaped[i][0])
            list_c = coordinates[Planet_n[i]]['y']
            list_c.append(y_shaped[i][1])
    
    return y_list, coordinates

In [ ]:
def Planets_Plot(Namen, AOrte, AGesch, Massen, G_val, Zeiten, Schritte):
    Planeten = {}
    for i in range(0, len(AOrte)):
        Planeten[Namen[i]] = {'pos' : AOrte[i], 'gesch': AGesch[i], 'masse': Massen[i]}
    t0 = Zeiten[0]
    tend = Zeiten[1]
    N = Schritte
    y_list, coordinates = Planeten_Coordinates(Planeten, G_val, t0, tend, N)
    for st in coordinates:
        r = np.random.rand()
        g = np.random.rand()
        b = np.random.rand()
        xs = coordinates[st]['x']
        ys = coordinates[st]['y']
        plt.plot(xs[0], ys[0], 'x', color=[r, g, b], label='Anfangspunkt'+st)
        plt.plot(xs, ys, color=[r, g, b], label=st)
        plt.plot(xs[-1], ys[-1], 'o', color=[r, g, b], label='Endpunkt'+st)

    plt.legend()
    return y_list, coordinates

In [ ]:
def Erhaltung(y):
    xs = range(0, len(y))
    Pe = []
    Ps = []
    P = []
    Le = []
    Ls = []
    L = []
    Ekin = []
    Epot = []
    E = []
    for y_ in y:
        o_s = y_[0:2]
        g_s = y_[2:4]
        o_e = y_[4:6]
        g_e = y_[6:8]
        
        r = (o_e-o_s)
        r_e = (m_s/(m_e+m_s))*r
        r_s = r-r_e
        
        #Impulserhaltung
        P_e = m_e*g_e
        Pe.append(P_e)
        P_e_g = P_e[0]+P_e[1]
        P_s = m_s*g_s
        Ps.append(P_s)
        P_s_g = P_s[0]+P_s[1]
        P.append(P_e_g+P_s_g)
        
        #Drehimpulserhaltung
        alpha_e = np.arccos(np.dot(g_e, r_e)/(LA.norm(g_e)*LA.norm(r_e)))
        L_e = m_e*LA.norm(g_e)*LA.norm(r_e)*math.sin(alpha_e)
        Le.append(L_e)
        alpha_s = np.arccos(np.dot(g_s, r_s)/(LA.norm(g_s)*LA.norm(r_s)))
        L_s = m_s*LA.norm(g_s)*LA.norm(r_s)*math.sin(alpha_s)
        Ls.append(L_s)
        L.append(L_e+L_s)
        
        #Energieerhaltung
        E_kin = m_e*((LA.norm(g_e))**2)/2+m_s*((LA.norm(g_s))**2)/2
        Ekin.append(E_kin)
        E_pot = -G_val*m_e*m_s/LA.norm(r)
        Epot.append(E_pot)
        E.append(E_kin+E_pot)

    Erg = {'x': xs, 'Impulserhaltung': [Pe, Ps, P], 'Drehimpulserhaltung': [Le, Ls, L], 'Energieerhaltung': [Ekin, Epot, E]}
    return Erg

In [ ]:
G_val = 6.674E-11

m_e = 5.972e24
a_ort_e = np.array([149.6e11, 0.])
a_gesch_e = np.array([0.0, 2.978e3])

m_s = 1.989e30
a_ort_s = np.array([0, 0.])
a_gesch_s = np.array([0., 0.])

In [ ]:
Namen = ['Sonne', 'Erde']
Orte = [a_ort_s, a_ort_e]
Gesch = [a_gesch_s, a_gesch_e]
Massen = [m_s, m_e]
ys, c = Planets_Plot(Namen, Orte, Gesch, Massen, G_val, [0, 3.154e10], 10000)
plt.axis('scaled')
plt.show()

In [ ]:
def Plot_Impulserhaltung(Erg):
    xs = Erg['x']
    imp = Erg['Impulserhaltung']
    Pe = imp[0]
    Ps = imp[1]
    P = imp[2]

    plt.plot(xs, np.array(Pe)[:, 0], color='#b7d5fd', label='Impuls in x Richtung Erde')
    plt.plot(xs, np.array(Pe)[:, 1], color='#004a85', label='Impuls in y Richtung Erde')
    plt.legend()
    plt.show()

    plt.plot(xs, np.array(Ps)[:, 0], color='#feff83', label='Impuls in x Richtung Sonne')
    plt.plot(xs, np.array(Ps)[:, 1], color='#e1e000', label='Impuls in y Richtung Sonne')
    plt.legend()
    plt.show()

    Pe_s = [Pe[i][0]+Pe[i][1] for i in range(0, len(Pe))]
    plt.plot(Pe_s, color='#0000ff', label='Impuls Erde')
    Ps_s = [Ps[i][0]+Ps[i][1] for i in range(0, len(Ps))]
    plt.plot(xs, Ps_s, color='#ffff00', label='Impuls Sonne')
    plt.legend()
    plt.show()

    plt.plot(xs, P, color='#00ff00')
    plt.title('Impuls gesamt')
    plt.show()

In [ ]:
def Plot_Drehimpulserhaltung(Erg):
    xs = Erg['x']
    dImp = Erg['Drehimpulserhaltung']
    Le = dImp[0]
    Ls = dImp[1]
    L = dImp[2]
    plt.plot(xs, Le, color='#0000ff')
    plt.title('Drehimpuls Erde')
    plt.show()
    plt.plot(xs, Ls, color='#ffff00')
    plt.title('Drehimpuls Sonne')
    plt.show()
    plt.plot(xs, L, color='#00ff00')
    plt.title('Drehimpuls gesamt')
    plt.show()

In [ ]:
def Plot_Energieerhaltung(Erg):
    xs = Erg['x']
    En = Erg['Energieerhaltung']
    Ekin = En[0]
    Epot = En[1]
    E = En[2]
    plt.plot(xs, Ekin, color='#ff00ff')
    plt.title('Kinetische Energie')
    plt.show()
    plt.plot(xs, Epot, color='#ff004e')
    plt.title('Potenzielle Energie')
    plt.show()
    plt.plot(xs, E, color='#00ff00')
    plt.title('Energie gesamt')
    plt.show()

In [ ]:
Erg = Erhaltung(ys)

In [ ]:
Plot_Impulserhaltung(Erg)

In [ ]:
Plot_Drehimpulserhaltung(Erg)

In [ ]:
Plot_Energieerhaltung(Erg)

In [ ]:
m_e = 5.972e24
a_ort_e = np.array([149.6e11, 0.])
a_gesch_e = np.array([0.0, 29.78e3])

m_s = 1.989e30
a_ort_s = np.array([0, 0.])
a_gesch_s = np.array([0., 0.])
G_val = 6.674E-11

In [ ]:
Namen = ['Sonne', 'Erde']
Orte = [a_ort_s, a_ort_e]
Gesch = [a_gesch_s, a_gesch_e]
Massen = [m_s, m_e]
ys, c = Planets_Plot(Namen, Orte, Gesch, Massen, G_val, [0, 2e9], 10000)

In [ ]:
Erg = Erhaltung(ys)

In [ ]:
Plot_Impulserhaltung(Erg)

In [ ]:
Plot_Drehimpulserhaltung(Erg)

In [ ]:
Plot_Energieerhaltung(Erg)

In [ ]:
def Bahnkurve_Animation(Anfangsorte, Anfangsgesch, massen, g, t0, tend, h):
    re = np.array(Anfangsorte[0])
    rs = np.array(Anfangsorte[1])
    ve = np.array(Anfangsgesch[0])
    vs = np.array(Anfangsgesch[1])
    
    scene = vp.canvas(title='Bahnkurve Erde', background=vp.color.black, forward=vec(0, 2, -1))
    erde = vp.sphere(pos=vec(re[0], re[1], 0), radius=1e12, color=vp.color.blue,  make_trail=True)
    sun = vp.sphere(pos=(vec(rs[0], rs[1], 0)), radius=2e12, color=vp.color.yellow)
    sunlight = vp.local_light(pos=vec(rs[0], rs[1], 0), color=vp.color.yellow)
    t = t0
    r_n = np.array([re[0], re[1], ve[0], ve[1], rs[0], rs[1], vs[0], vs[1]])
    dr_n = ODEPlaneten_rk(t0, r_n, massen, g)
    while t <= tend:
        vp.rate(5000)
        r_n = rk4(r_n, t, massen, g, h, ODEPlaneten_rk)
        erde.pos = vec(r_n[0], r_n[1], 0) 
        sun.pos = vec(r_n[4], r_n[5], 0)
        sunlight.pos = vec(r_n[4], r_n[5], 0)
        t += h

In [ ]:
m_e = 5.972e24
a_ort_e = np.array([149.6e11, 0.])
a_gesch_e = np.array([0.0, 29.78e3])

m_s = 1.989e30
a_ort_s = np.array([0, 0.])
a_gesch_s = np.array([0., 0.])
G_val = 6.674E-11

In [ ]:
Bahnkurve_Animation([a_ort_e, a_ort_s], [a_gesch_e, a_gesch_s], [m_e, m_s], G_val, 0, 2e9, 1e5)

In [ ]:
G_val = 6.674E-11

m_e = 5.972e24
a_ort_e = np.array([149.6e11, 0.])
a_gesch_e = np.array([0.0, 2.978e3])

m_s = 1.989e30
a_ort_s = np.array([0, 0.])
a_gesch_s = np.array([0., 0.])

In [ ]:
Bahnkurve_Animation([a_ort_e, a_ort_s], [a_gesch_e, a_gesch_s], [m_e, m_s], G_val, 0, 3.145e10, 1e6)

### 3.
Überprüfen Sie die Impuls-, Drehimpuls- und Energieerhaltung analytisch. Verwenden Sie dazu Polarkoordinaten und leinten Sie die Bewegungsgleichung in Polarkoordinaten her:
$$ m(\ddot{r} - r*\dot{\theta}^2) = -G\frac{mM}{r^2}$$
$$ m(r\ddot{\theta}+2\dot{r}\dot{\theta}) = 0 $$

### 4.
Varieeren Sie die Anfangsgeschwindigkeit der Erde und beschreiben Sie die verschiedenen Bahnformen, die sich ergeben.

### 5.
Beschrieben Sie mit Hilfe des Programms die Bewegung zweier Fast gleich großer Massen (Zweikörperproblem). Denken Sie an die Impulserhaltung!

### 6.
Drei Sterne seinen anfangs in Ruhe mit folgenden Massen und Positionen (arbitrary units):

|       | Masse |  x |  y |
|:-------:|:-------:|:----:|:----:|
|**Stern 1**|  150  |  3 |  1 |
|**Stern 2**|  200  | -1 | -2 |
|**Stern 3**|  250  | -1 |  1 |

(Die drei Sterne seien in der $xy$-Ebene.)

#### (a)
Zeigen Sie, dass die Bewegungsgleichung für Position $\vec{r}_1$ des ersten Sterns gegeben ist durch:
$$\ddot{\vec{r}}_1 = Gm_2\frac{\vec{r}_2-\vec{r}_1}{|\vec{r}_2-\vec{r}_1|^3} + Gm_3\frac{\vec{r}_3-\vec{r}_1}{|\vec{r}_3-\vec{r}_1|^3} $$
und stellen Sie die Gleichung für $\vec{r}_2$ und $\vec{r}_3$ der beiden anderen Sterne auf.

$$ \vec{F}_{ij} = -G\frac{m_im_j}{r^2}\vec{e}_r $$
$$ r = |\vec{r}_j - \vec{r}_i| $$
$$ \vec{e}_r = \frac{\vec{r}_j - \vec{r}_i}{|\vec{r}_j - \vec{r}_i|}$$

$$ \vec{F}_{ij} = -G\frac{m_im_j}{|\vec{r}_j - \vec{r}_i|^2}\frac{\vec{r}_j - \vec{r}_i}{|\vec{r}_j - \vec{r}_i|} = -Gm_im_j\frac{\vec{r}_j - \vec{r}_i}{|\vec{r}_j - \vec{r}_i|^3}$$

$$ m_i\ddot{\vec{r}}_i = \sum_{j\neq i} F_{ij} = \sum_{j\neq i} -Gm_im_j\frac{\vec{r}_j - \vec{r}_i}{|\vec{r}_j - \vec{r}_i|^3}$$

$$ \ddot{\vec{r}}_i = \sum_{j\neq i} -Gm_j\frac{\vec{r}_j - \vec{r}_i}{|\vec{r}_j - \vec{r}_i|^3}$$

$$ \ddot{\vec{r}}_1 = -Gm_2\frac{\vec{r}_2 - \vec{r}_1}{|\vec{r}_2 - \vec{r}_1|^3} -Gm_3\frac{\vec{r}_3 - \vec{r}_1}{|\vec{r}_3 - \vec{r}_1|^3}$$
$$ \ddot{\vec{r}}_2 = -Gm_1\frac{\vec{r}_1 - \vec{r}_2}{|\vec{r}_1 - \vec{r}_2|^3} -Gm_3\frac{\vec{r}_3 - \vec{r}_2}{|\vec{r}_3 - \vec{r}_2|^3}$$
$$ \ddot{\vec{r}}_3 = -Gm_1\frac{\vec{r}_1 - \vec{r}_3}{|\vec{r}_1 - \vec{r}_3|^3} -Gm_2\frac{\vec{r}_2 - \vec{r}_3}{|\vec{r}_2 - \vec{r}_3|^3}$$

#### (b)
Setzten Sie $G=1$ und schreiben sie ein Programm, das die Bewegungsgleichungen löst. Plotten Sie $y$ als Funktion von $x$ und animieren Sie die Bewegung. Die Sterne bewegen sich sehr schnell, wenn sie sich nähern und sind sehr langsam, wenn sie weit voneinander entfernt sind. Eine adaptive Methode könnte deshab nützlich sein.

In [2]:
G = 1
Sterne = {'Stern1': {'m' : 150, 'pos': [3, 1]}, 'Stern2': {'m' : 200, 'pos': [-1, -2]},'Stern3': {'m' : 250, 'pos': [-1, 1]}}

In [3]:
def ODEsterne(y, t):
    '''
    berechnet die rechte Seite der Differentialgleichung 
    Zunächst kommen alle Variablen (x,y,vx,vy) für den ersten Planeten, dann alle 4 für den zweiten,... 
    :param t: aktueller zeitpunkt
    :param y: aktueller y-Wert (n Komponenten)
    :return: rechte Seite f der Dgl. y'=f(t,y)
    '''

    # Daten der Planeten
    G = 1  # Gravitationskonstante
    #masse_Sterne = [150, 200, 250]
    #print('Massen', Massen)
    y = np.array(y)
    y = y.reshape(-1, 4)
    #print(y)
    rs = []
    dy = []
    #print(planets)
    for i in range(0, len(y)):
        F_i = 0
        rs.append(np.array([y[i][0], y[i][1]]))
        r_i = np.array([y[i][0], y[i][1]])
        for j in range(0, len(y)):
            if i != j:
                r_j = np.array([y[j][0], y[j][1]])
                m_j = Massen[j]
                dist = r_i-r_j
                F_i += m_j *((dist)/(LA.norm(dist))**3)
        ddr_i = -G*F_i
        dy.append(y[i][2])
        dy.append(y[i][3])
        dy.append(ddr_i[0])
        dy.append(ddr_i[1])
    return dy

In [4]:
def Sterne_Coordinates(Sterne, t0, tend, N):
    Stern_n = []
    tvals, h = np.linspace(t0, tend, N, retstep=True)
    #print(h)
    coordinates = {}
    y_sterne = []
    for stern in Sterne:
        Stern_n.append(stern)
        x_s = Sterne[stern]['pos'][0]
        y_s = Sterne[stern]['pos'][1]
        vx_s = Sterne[stern]['gesch'][0]
        vy_s = Sterne[stern]['gesch'][1]
        y_sterne.append(x_s)
        y_sterne.append(y_s)
        y_sterne.append(vx_s)
        y_sterne.append(vy_s)
        coordinates[stern] = {'x':[], 'y':[]}
    y_list = [y_sterne]
    y_list = integrate.odeint(ODEsterne, y_sterne, tvals)
    y_list[0] = np.array(y_list[0])
    for y_step in y_list:
        y_shaped = y_step.reshape(-1, 4)
        for i in range(0, len(y_shaped)):
            list_c = coordinates[Stern_n[i]]['x']
            list_c.append(y_shaped[i][0])
            list_c = coordinates[Stern_n[i]]['y']
            list_c.append(y_shaped[i][1])
    
    return coordinates

In [5]:
def Sterne_Plot(Sterne, t0, tend, N):
    coordinates = Sterne_Coordinates(Sterne, t0, tend, N)
    for st in coordinates:
        r = np.random.rand()
        g = np.random.rand()
        b = np.random.rand()
        xs = coordinates[st]['x']
        ys = coordinates[st]['y']
        #plt.plot(xs[0], ys[0], 'o', color=[r, g, b])
        plt.plot(xs, ys, color=[r, g, b], label=st)
        plt.plot(xs[-1], ys[-1], 'o', color=[r, g, b])
        plt.plot(xs[0], ys[0], 'x', color=[r, g, b])

    plt.legend()
    plt.show()

In [ ]:
s1 = {'m': 150, 'pos': [ 3,  1], 'gesch': [0, 0]}
s2 = {'m': 200, 'pos': [-1, -2], 'gesch': [0, 0]}
s3 = {'m': 250, 'pos': [-1,  1], 'gesch': [0, 0]}
Sterne = {'Stern1': s1, 'Stern2': s2, 'Stern3': s3}
Massen = [150, 200, 250]

In [ ]:
Sterne_Plot(Sterne, 0, 5, 10000)

In [ ]:
s1 = {'m': 10, 'pos': [ 0,  2], 'gesch': [-0.4, -0.4]}
s2 = {'m': 200, 'pos': [ 1,  1], 'gesch': [-3, -3]}
s3 = {'m': 200, 'pos': [-1,  1], 'gesch': [3, 3]}
Sterne = {'Stern1': s1, 'Stern2': s2, 'Stern3': s3}
Massen = [1, 200, 200]


In [ ]:
Sterne_Plot(Sterne, 0, 2.4, 100000)

In [ ]:
s1 = {'m': 0.1, 'pos': [ 0,  2], 'gesch': [0, 0]}
s2 = {'m': 200, 'pos': [ 1,  1], 'gesch': [0, -90]}
s3 = {'m': 20000, 'pos': [0,  1], 'gesch': [0, 0]}
Sterne = {'Stern1': s1, 'Stern2': s2, 'Stern3': s3}
Massen = [10, 200, 20000]

In [ ]:
Sterne_Plot(Sterne, 0, 0.1, 100000)

In [ ]:
s1 = {'m': 150, 'pos': [ 3,  1], 'gesch': [2, 0]}
s2 = {'m': 200, 'pos': [-1, -2], 'gesch': [0, 2]}
s3 = {'m': 250, 'pos': [-1,  1], 'gesch': [-1, 0]}
Sterne = {'Stern1': s1, 'Stern2': s2, 'Stern3': s3}
Massen = [150, 200, 250]

In [ ]:
Sterne_Plot(Sterne, 0, 3.5, 100000)

In [6]:
def Sterne_Animation(Sterne, t0, tend, N, Rate):
    
    scene = vp.canvas(title='Animation Sterne', background=vp.color.black, forward=vec(0, 2, -1))
    stern1 = vp.sphere(pos=vec(Sterne['Stern1']['pos'][0], Sterne['Stern1']['pos'][1], 0), radius=0.1, color=vp.color.blue, make_trail=True)
    stern2 = vp.sphere(pos=vec(Sterne['Stern2']['pos'][0], Sterne['Stern2']['pos'][1], 0), radius=0.1, color=vp.color.green, make_trail=True)
    stern3 = vp.sphere(pos=vec(Sterne['Stern3']['pos'][0], Sterne['Stern3']['pos'][1], 0), radius=0.1, color=vp.color.yellow, make_trail=True)
    coordinates = Sterne_Coordinates(Sterne, t0, tend, N)
    h = abs(tend-t0)/N
    t = t0
    i = 0
    #y_n = ODEsterne_rk(t0, y_sterne)
    #while True:
    while i < len(coordinates['Stern1']['x']):
    #while True:
        #print(y_n)
        s1 = coordinates['Stern1']
        s2 = coordinates['Stern2']
        s3 = coordinates['Stern3']
        vp.rate(Rate)
        stern1.pos = vec(s1['x'][i], s1['y'][i],0)
        #vp.rate(500)
        stern2.pos = vec(s2['x'][i], s2['y'][i],0)
        #vp.rate(500)
        stern3.pos = vec(s3['x'][i], s3['y'][i],0)
    #    t += h
        i += 1
#Sterne_Animation(Sterne, 0, 10, 0.05)

In [7]:
s1 = {'m': 150, 'pos': [ 3,  1], 'gesch': [0, 0]}
s2 = {'m': 200, 'pos': [-1, -2], 'gesch': [0, 0]}
s3 = {'m': 250, 'pos': [-1,  1], 'gesch': [0, 0]}
Sterne = {'Stern1': s1, 'Stern2': s2, 'Stern3': s3}
Massen = [150, 200, 250]

In [8]:
Sterne_Animation(Sterne, 0, 3, 100000, 1000)

<IPython.core.display.Javascript object>

In [9]:
s1 = {'m': 10, 'pos': [ 0,  2], 'gesch': [-0.4, -0.4]}
s2 = {'m': 200, 'pos': [ 1,  1], 'gesch': [-3, -3]}
s3 = {'m': 200, 'pos': [-1,  1], 'gesch': [3, 3]}
Sterne = {'Stern1': s1, 'Stern2': s2, 'Stern3': s3}
Massen = [1, 200, 200]

In [11]:
Sterne_Animation(Sterne, 0, 4, 100000, 500)

<IPython.core.display.Javascript object>

ERROR! Session/line number was not unique in database. History logging moved to new session 311


In [35]:
s1 = {'m': 2, 'pos': [ 0,  1], 'gesch': [0, 9]}
s2 = {'m': 2, 'pos': [ 1,  1], 'gesch': [0, 7]}
s3 = {'m': 200, 'pos': [-1,  1], 'gesch': [0, 0]}
Sterne = {'Stern1': s1, 'Stern2': s2, 'Stern3': s3}
Massen = [2, 2, 200]

ERROR! Session/line number was not unique in database. History logging moved to new session 325


In [36]:
Sterne_Animation(Sterne, 0, 1, 100000, 900)

<IPython.core.display.Javascript object>